# Ames Housing EDA
This notebook explores the Ames Housing dataset to identify trends, relationships, and data quality issues in preparation for feature engineering and modeling.

In [173]:
# NOTE: Dropped rows with missing values in low-null columns (1–2 missing total)
#       Will revisit this strategy if we choose to impute later on

### Import Statements

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load Data

In [175]:
df = pd.read_csv('../data/AmesHousing.csv')

# Create a copy of preprocessed data
df_og = df.copy()

### Initial Overview

In [176]:
print(df.shape)
print(df.info())
print(df.head())

(2930, 82)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  Hou

### Check for Duplicate and Missing Values

In [177]:
# Duplicates
print("Number of duplicate rows:", df.duplicated().sum(), "\n")

# Missing Values
print("Number of missing values per column (non-zero):")
num_missing = df.isnull().sum()
print(num_missing[num_missing > 0].to_string())

# Remove rows corresponding to columns that have only 1 or 2 missing values
low_null_cols = df.columns[df.isnull().sum().between(1, 2)]
df = df.dropna(subset=low_null_cols)

# Print unique non-null values for other columns with missing values
high_null_cols = df.columns[df.isnull().sum() > 2]
for col in high_null_cols:
    print(f"\n{col} - missing: {df[col].isnull().sum()}")
    print(df[col].dropna().unique())

# Validate if null values should be filled
print(df[df['Fireplace Qu'].isnull()]['Fireplaces'].value_counts())
print(df[df['Pool QC'].isnull()]['Pool Area'].value_counts())
print(df[df['Misc Feature'].isnull()]['Misc Val'].value_counts())
print(df[df['Mas Vnr Type'].isnull()][['Mas Vnr Area']].value_counts())
print(df[df['Garage Type'].isnull()]['Garage Area'].value_counts())
# Get rows where Garage Type is NOT null but at least one other garage col IS null
garage_cols = ['Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond']
mask = df['Garage Type'].notnull() & df[garage_cols].isnull().any(axis=1)
print(df[mask][['Garage Type'] + garage_cols + ['Garage Cars', 'Garage Area']])
# Define basement-related columns and identify rows with partially missing basement info
basement_cols = ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
partial_missing_basement = df[basement_cols].isnull().any(axis=1) & ~df[basement_cols].isnull().all(axis=1)

Number of duplicate rows: 0 

Number of missing values per column (non-zero):
Lot Frontage       490
Alley             2732
Mas Vnr Type      1775
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin SF 1         1
BsmtFin Type 2      81
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual        159
Garage Cond        159
Pool QC           2917
Fence             2358
Misc Feature      2824

Lot Frontage - missing: 490
[141.  80.  81.  93.  74.  78.  41.  43.  39.  60.  75.  63.  85.  47.
 152.  88. 140. 105.  65.  70.  26.  21.  53.  24. 102.  98.  83.  94.
  95.  90.  79. 100.  44. 110.  61.  36.  67. 108.  59.  92.  58.  56.
  73.  72.  84.  76.  50.  55.  68. 107.  25.  30.  57.  40

### Treat Missing Values

In [180]:
# Replace null values with 'None' or 0 where appropriate
df['Fireplace Qu'] = df['Fireplace Qu'].fillna('None')
df['Pool QC'] = df['Pool QC'].fillna('None')
df['Fence'] = df['Fence'].fillna('None')
df['Misc Feature'] = df['Misc Feature'].fillna('None')
df['Alley'] = df['Alley'].fillna('None')
# Set Mas Vnr Type to 'None' only if area is 0, otherwise 'Unknown'
# and drop the row if Mas Vnr Type and Mas Vnr Area are both unknown
df.loc[(df['Mas Vnr Type'].isnull()) & (df['Mas Vnr Area'] == 0), 'Mas Vnr Type'] = 'None'
df.loc[(df['Mas Vnr Type'].isnull()) & (df['Mas Vnr Area'] != 0), 'Mas Vnr Type'] = 'Unknown'
df = df[~((df['Mas Vnr Type'] == 'Unknown') & (df['Mas Vnr Area'].isnull()))]
# Fill remaining garage-related missing values
# and drop the one garage row with missing qualitative info
garage_fill_cols = ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']
df[garage_fill_cols] = df[garage_fill_cols].fillna('None')
df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(0)
# Drop basement rows with partial missing info and fill rest
basement_cols = ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
partial_missing_basement = df[basement_cols].isnull().any(axis=1) & ~df[basement_cols].isnull().all(axis=1)
df = df[~partial_missing_basement]
# Impute Lot Frontage with neghborhood median and drop remaining rows
df['Lot Frontage'] = df.groupby('Neighborhood')['Lot Frontage'].transform(
    lambda x: x.fillna(x.median()) if x.notna().sum() > 0 else x
)